In [1]:
import datetime as dt
from odds_and_other_projections import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def _calculate_odds(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))*100
    if odds>0:
        return (100/(odds+100))*100

In [3]:
 # Team map
team_map = {
'CHC' : 'Cubs',
'STL' : 'Cardinals',
'MIN' : 'Twins',
'DET' : 'Tigers',
'BOS' : 'Red Sox',
'TOR' : 'Blue Jays',
'SEA' : 'Mariners',
'HOU' : 'Astros',
'SD' : 'Padres',
'OAK' : 'Athletics',
'BAL' : 'Orioles',
'MIA' : 'Marlins',
'PHI' : 'Phillies',
'WSH' : 'Nationals',
'NYM' : 'Mets',
'ATL' : 'Braves',
'TB' : 'Rays',
'NYY' : 'Yankees',
'TEX' : 'Rangers',
'ARI' : 'Diamondbacks',
'CHC' : 'Cubs',
'CIN' : 'Reds',
'KC' : 'Royals',
'CWS' : 'White Sox',
'LAA' : 'Angels',
'COL' : 'Rockies',
'SF' : 'Giants',
'LAD' : 'Dodgers',
'MIL' : 'Brewers',
'PIT' : 'Pirates',
'CLE' : 'Guardians'
}

# Instantiating webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.actionnetwork.com/mlb/odds')

# Navigating to ML
ml_button = driver.find_element_by_xpath("//*[@id='__next']/div/main/div/div[2]/div/div[1]/div[2]/select")
select = Select(ml_button)
select.select_by_visible_text('Moneyline')

# Getting odds table and formatting
html = driver.page_source
tables = pd.read_html(html)
odds = tables[0]
odds = odds.iloc[::2]
odds.reset_index(drop = True, inplace = True)

# Iterating through to get home/away and odds
odds_df = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Home_Odds', 'Away_Odds'])
for index, row in odds.iterrows():
    # Retreiving home and away teams
    teams = {}
    for key in team_map.keys():
        if row.Scheduled.find(key) != -1:
            teams[row.Scheduled.find(key)] = key
    keys = []
    for key in teams.keys():
        keys.append(key)
    if keys[0] > keys[1]:
        home_team = teams[keys[0]]
        away_team = teams[keys[1]]
    else:
        home_team = teams[keys[1]]
        away_team = teams[keys[0]]
    # Retreiving odds
    ml_string = row['Unnamed: 5']
    if len(ml_string) == 8:
        ml_away = ml_string[:4]
        ml_home = ml_string[-4:]
    if len(ml_string) == 9:
        if (ml_string[4] == '+') | (ml_string[4]=='-'):
            ml_away = ml_string[:4]
            ml_home = ml_string[-5:]
        else:
            ml_away = ml_string[:5]
            ml_home = ml_string[-4:]
    if len(ml_string) == 10:
            ml_away = ml_string[:5]
            ml_home = ml_string[-5:]
    try:
        ml_away = float(ml_away)
    except:
        continue
    try:
        ml_home = float(ml_home)
    except:
        continue
    series = pd.Series([home_team, away_team, ml_home, ml_away], index = odds_df.columns)
    odds_df = odds_df.append(series, ignore_index = True)
odds_df['Home_Prob'] = odds_df.Home_Odds.apply(_calculate_odds)
odds_df['Away_Prob'] = odds_df.Away_Odds.apply(_calculate_odds)
odds_df['Home_Team'] = odds_df.Home_Team.apply(lambda x: team_map[x])
odds_df['Away_Team'] = odds_df.Away_Team.apply(lambda x: team_map[x])
return odds_df



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Get LATEST driver version for 100.0.4896
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


In [4]:
odds_df

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Cardinals,Mets,104.0,-122.0,49.019608,54.954955
1,White Sox,Royals,-178.0,150.0,64.028777,40.000000
2,Diamondbacks,Dodgers,164.0,-196.0,37.878788,66.216216
3,Pirates,Brewers,152.0,-180.0,39.682540,64.285714
4,Reds,Padres,156.0,-186.0,39.062500,65.034965
5,Rays,Mariners,-138.0,118.0,57.983193,45.871560
6,Phillies,Rockies,-225.0,188.0,69.230769,34.722222
7,Nationals,Marlins,122.0,-144.0,45.045045,59.016393
8,Yankees,Orioles,-250.0,205.0,71.428571,32.786885
9,Blue Jays,Red Sox,-142.0,120.0,58.677686,45.454545


In [ ]:
# Function to calculate potential payoffs
def calculate_payoff_external(row):
    if row.Bet_538<=0:
        payoff = 0
    else:
        if row.Home_KC_538>0:
            if row.Home_Odds>0:
                payoff = (row.Home_Odds/100)*row.Bet_538
            if row.Home_Odds<0:
                payoff = row.Bet_538/((abs(row.Home_Odds)/100))
        if row.Away_KC_538>0:
            if row.Away_Odds>0:
                payoff = (row.Away_Odds/100)*row.Bet_538
            if row.Away_Odds<0:
                payoff = row.Bet_538/((abs(row.Away_Odds)/100))
    return payoff

# Getting yesterday's results from CBS
link = 'https://www.cbssports.com/mlb/scoreboard/' + yesterday_string + '/'
tables = pd.read_html(link)
results_table = pd.DataFrame(columns = ['Home_Team', 'Away_Team', 'Winner'])
for table in tables:
    if list(table.columns) == ['Unnamed: 0', 'R', 'H', 'E']:
        # Getting team names
        team_away_list = table.iloc[0,0].split(' ')
        del team_away_list[-2:]
        if len(team_away_list) == 2:
            team_away = team_away_list[0] + ' ' + team_away_list[1]
        else:
            team_away = team_away_list[0]
        team_home_list = table.iloc[1,0].split(' ')
        del team_home_list[-2:]
        if len(team_home_list) == 2:
            team_home = team_home_list[0] + ' ' + team_home_list[1]
        else:
            team_home = team_home_list[0]
        # Getting score and determining winner
        runs_away = table.iloc[0,1]
        runs_home = table.iloc[1,1]
        if runs_away>runs_home:
            winner = team_away
        else:
            winner = team_home
        # Appending to results table
        series = pd.Series([team_home, team_away, winner], index = results_table.columns)
        results_table = results_table.append(series, ignore_index = True)        
    else:
        continue
for column in list(results_table.columns):
    results_table[column] = results_table[column].apply(lambda x: team_map[x])

# Reading in yesterdays bets and creating tracker columns
yesterdays_bets = pd.read_csv('past_bets/external/bets_external_' + yesterday_string + '.csv', index_col = 0)
yesterdays_bets = yesterdays_bets[(yesterdays_bets.Home_Team != '538') & (yesterdays_bets.Away_Team != '538')]
yesterdays_bets['Home_Team'] = yesterdays_bets['Home_Team'].apply(lambda x: team_map[x])
yesterdays_bets['Away_Team'] = yesterdays_bets['Away_Team'].apply(lambda x: team_map[x])
yesterdays_bets = yesterdays_bets[yesterdays_bets.Bet_538>0]
yesterdays_bets.reset_index(drop = True, inplace = True)
yesterdays_bets['Won_538'] = 0
yesterdays_bets['Tracker_538'] = 0
for index, row in yesterdays_bets.iterrows():
    home_team = row.Home_Team
    away_team = row.Away_Team
    payoff_538 = calculate_payoff_external(row, Fivethirtyeight = True)
    if row.Bet_538>0:
        if (row.Home_KC_538>0) & (home_team in results_table['Winner'].values):
            yesterdays_bets.loc[index, 'Won_538'] = 1
        elif (row.Away_KC_538>0) & (away_team in results_table['Winner'].values):
            yesterdays_bets.loc[index, 'Won_538'] = 1
        else:
            yesterdays_bets.loc[index, 'Won_538'] = 0
    else:
        yesterdays_bets.loc[index, 'Won_538'] = -1
    if yesterdays_bets.loc[index, 'Won_538'] == 1:
        if index == 0:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 + payoff_538
        else:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] + payoff_538
    else:
        if index == 0:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_capital_538 - row.Bet_538
        else:
            yesterdays_bets.loc[index, 'Tracker_538'] = yesterdays_bets.loc[(index-1), 'Tracker_538'] - row.Bet_538
        
return yesterdays_bets